# TP4: Estimación de peso y dimensiones de los envíos de Mercado Libre

# Materia: Aprendizaje supervisado

## Análisis del dataset. Comunicación de resultados y conclusiones

A partir de lo visto en la teoría de la materia y del cuarto laboratorio, diagramar una comunicación en formato textual o interactivo describiendo la solución de las actividades propuestas a continuación. Al final de las mismas se proveen actividades opcionales (no obligatorias) que pueden resultar de interés.

### Actividades Propuestas:

    1. Splitear el dataset en train/test (80-20). Recordar la utilidad train_test_split de scikit-learn y utilizar los parámetros `random_state` y `stratify` y explicar su función. El target en este práctico será múltiple: SHP_WEIGHT , SHP_LENGTH , SHP_HEIGHT y SHP_WIDTH . Esto significa que los modelos deberán predecir 4 valores en simultáneo en vez de 1.
    
    2. Entrenar y evaluar con al menos 3 nuevos modelos (Sugerencias: SVR , RandomForestRegressor, GradientBoostingRegressor, etc.) Obligatorio: Probar con una red neuronal. Puede ser de scikit-learn o de alguna otra librería que deseen como keras , pytorch , etc.). Junto con las métricas debe entregarse una breve descripción de cómo funciona cada modelo. Importante: Para evaluar, por ejemplo mean_absolute_error provee un parámetro multioutput que debería tomar el valor ` raw_values ` para reportar métricas para cada dimension de output por separado.
    
    3. Para estos nuevos modelos tunear hiper-parámetros. Para las evaluaciones utilizar la técnica de k-fold cross-validation (ver cross-validation ) y explicar los resultados.

    4. Elegir el mejor modelo entrenado hasta el momento según f1-score. Comparar las métricas de este modelo vs. las métricas de evaluar 4 modelos por separado, un modelo para cada uno de los targets. Interpretar los resultados. 
    
La comunicación debe estar apuntada a un público técnico pero sin conocimiento del tema particular, como por ejemplo, sus compañeros de clase o stakeholders del proyecto. Idealmente, además del documento se debería generar una presentación corta para stakeholders explicando el análisis realizado sobre los datos y las conclusiones obtenidas de tal análisis.
    
Se evaluarán los siguientes aspectos:

    ● El informe debe contener un mensaje claro y presentado de forma concisa.
    ● Los gráficos deben aplicar los conceptos de percepción visual vistos en clase.
    ● Se debe describir o estimar la significancia estadística de su trabajo.


## Carga de datos

In [4]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
import seaborn
import scipy as sc
from math import sqrt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from ast import literal_eval
from pandas.io.json import json_normalize
from fancyimpute import KNN
#extras para TP4
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor 
#import tensorflow as tf
%matplotlib inline

Using TensorFlow backend.


In [2]:
#from visualization import plot_confusion_matrix, plot_learning_curve
#import os
#import sys
#sys.path.append(os.getcwd())
#from ml.visualization import plot_confusion_matrix, plot_learning_curve


In [5]:
random.seed(0)
DATASET = '../meli_dataset_20190426.csv'
df_original = pd.read_csv(DATASET, low_memory=False)

In [6]:
df= df_original
df = df.head(10000)

## Preprocesamiento:  

En base a lo desarrollado en el TP2 se eliminan los registros con `STATUS` 404 o con faltantes en la variables `SHP`, se agrupa por `ITEM_ID` y se reemplaza por la mediana. Además, se codifican algunas variables categóricas y se imputan valores a los faltantes de la variable `PRICE`.

In [7]:
# Eliminación de registros con status 404
df = df[df.STATUS != "404"]
df = df.drop(columns=['STATUS'])
df.sample(5)

# Eliminación de registros con faltantes en las variables SHP 
df = df.dropna(subset=['SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', 'SHP_HEIGHT'])

# Agrupación por item id y reemplazo por mediana
# Agrupamos por item_id
df_grouped = df.groupby(['ITEM_ID'], as_index=False).median()
#Ordenamos el dataframe por item_id
df.sort_values('ITEM_ID', inplace = True)
# Eliminamos filas con item_id duplicados
df.drop_duplicates(subset='ITEM_ID', keep=False, inplace=True)
# Actualizamos dataframe original con la mediana de pesos y medidas
df.set_index('ITEM_ID', inplace=True)
df.update(df_grouped.set_index('ITEM_ID', inplace=True))
df.reset_index()

# Binarización de CATALOG_PRODUCT_ID, CONDITION y DOMAIN_ID

column = 'CATALOG_PRODUCT_ID'
lb = LabelBinarizer()
lb_results = lb.fit_transform(df[column])
#pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_)).head(10)
CATALOG_PRODUCT_ID_ENCODED = pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_))

column = "CONDITION"
lb = LabelBinarizer()
lb_results = lb.fit_transform(df[column].astype(str))
pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_)).head(10)
CONDITION_ENCODED = pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_))

column = 'DOMAIN_ID'
lb = LabelBinarizer()
lb_results = lb.fit_transform(df[column].astype(str))
#pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_)).head(10)
DOMAIN_ID_ENCODED = pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_))
# Pegado de las variables categoricas codificadas al dataset
df["id"]=CONDITION_ENCODED.index
df=df.set_index("id")
df = pd.concat([df,CATALOG_PRODUCT_ID_ENCODED, CONDITION_ENCODED, DOMAIN_ID_ENCODED], axis=1)


In [8]:
df

,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,...,DOMAIN_ID_MLB-WIRELESS_ANTENNAS,DOMAIN_ID_MLB-WIRELESS_CHARGERS,DOMAIN_ID_MLB-WIRELESS_FM_TRANSMITTERS,DOMAIN_ID_MLB-WIRE_STRIPPERS,DOMAIN_ID_MLB-WOMEN_SWIMWEAR,DOMAIN_ID_MLB-WRENCHES,DOMAIN_ID_MLB-WRENCH_SETS,DOMAIN_ID_MLB-WRISTWATCHES,DOMAIN_ID_MLB-XENON_KITS,DOMAIN_ID_nan
0,775.0,50.0,20.0,10.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-ENGINE_GASKET_SETS,750.00,QD3YJ9751S,...,0,0,0,0,0,0,0,0,0,0
1,6100.0,70.0,25.0,5.0,"[{'id': 'BEDDING_SET_SIZE', 'name': 'Tamanho',...",H53U1H7Q5G,new,MLB-BEDDING_SETS,119.90,J3EY3QAB29,...,0,0,0,0,0,0,0,0,0,0
2,464.0,20.0,11.0,10.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-AUTOMOBILE_FUEL_PUMPS,349.90,NO4W1R9S3D,...,0,0,0,0,0,0,0,0,0,0
3,150.0,25.0,25.0,11.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-PENDRIVES,21.99,KIQX6YQZI4,...,0,0,0,0,0,0,0,0,0,0
4,3719.0,42.0,34.0,13.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",GITRVCM7WO,used,MLB-GAME_CONSOLES,849.00,ZQIKYCCZ7E,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3987,431.0,25.0,25.0,5.0,"[{'id': 'CLOSING', 'name': 'Fecho', 'value_id'...",H53U1H7Q5G,new,MLB-FANNY_PACKS,69.90,GPWP5IFQEN,...,0,0,0,0,0,0,0,0,0,0
3988,150.0,20.0,20.0,20.0,"[{'id': 'ITEM_CONDITION', 'name': 'Condição do...",H53U1H7Q5G,new,MLB-PORTABLE_ELECTRIC_MASSAGERS,7.50,OFLRK20BUP,...,0,0,0,0,0,0,0,0,0,0
3989,3880.0,36.0,24.0,13.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-ENGINE_OILS,145.90,MQICEHKRH5,...,0,0,0,0,0,0,0,0,0,0
3990,1040.0,28.0,18.0,8.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",CCNZQYJ1G6,new,MLB-ROUTERS,329.49,ANYX5441IO,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Imputación de faltantes de PRICE por KNN
df_numeric = df.select_dtypes([np.number])
df_filled = pd.DataFrame(KNN(3).fit_transform(df_numeric))
df_filled.columns=df_numeric.columns
df=df_filled

Imputing row 1/3992 with 0 missing, elapsed time: 209.165
Imputing row 101/3992 with 0 missing, elapsed time: 209.175
Imputing row 201/3992 with 0 missing, elapsed time: 209.182
Imputing row 301/3992 with 0 missing, elapsed time: 209.189
Imputing row 401/3992 with 0 missing, elapsed time: 209.196
Imputing row 501/3992 with 0 missing, elapsed time: 209.202
Imputing row 601/3992 with 0 missing, elapsed time: 209.209
Imputing row 701/3992 with 0 missing, elapsed time: 209.217
Imputing row 801/3992 with 0 missing, elapsed time: 209.227
Imputing row 901/3992 with 0 missing, elapsed time: 209.235
Imputing row 1001/3992 with 0 missing, elapsed time: 209.242
Imputing row 1101/3992 with 0 missing, elapsed time: 209.250
Imputing row 1201/3992 with 0 missing, elapsed time: 209.257
Imputing row 1301/3992 with 0 missing, elapsed time: 209.265
Imputing row 1401/3992 with 1 missing, elapsed time: 209.273
Imputing row 1501/3992 with 0 missing, elapsed time: 209.280
Imputing row 1601/3992 with 0 missin

In [10]:
df=df_filled

In [9]:
# Probablemente no se relize la transformación logarítmica, por ahora se comenta esta perte del código
#np.log(df["SHP_WEIGHT"])
#df["SHP_WEIGHT_LOG"] = np.log(df["SHP_WEIGHT"])
#df= df.drop(columns=["SHP_WEIGHT", "SHP_LENGTH", "SHP_WIDTH", "SHP_HEIGHT",  "CATALOG_PRODUCT_ID_A0RY70BE19", 'CONDITION_nan', "DOMAIN_ID_nan"])

# Actividad 1:    

Splitear el dataset en train/test (80-20). Recordar la utilidad train_test_split de scikit-learn y utilizar los parámetros `random_state` y `stratify` y explicar su función. El target en este práctico será múltiple: SHP_WEIGHT , SHP_LENGTH , SHP_HEIGHT y SHP_WIDTH . Esto significa que los modelos deberán predecir 4 valores en simultáneo en vez de 1.

In [11]:
# división entre instancias y etiquetas

X, y = df.iloc[:, 4:], df[['SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', 'SHP_HEIGHT']]

# división entre entrenamiento y evaluación
#stratify=y no se emplea porque no es problema de clasificación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

El parámetro stratify permite realizar un muestreo estratificado por clases del dataset. De este modo se obtienen conjuntos de entrenamiento y de testing que tienen la misma proporción de cada clase de la etiqueta que el dataset original (si estratificamos de acuerdo a la la etiqueta y). Al tratarse de un problema de regresión entendemos que no tiene sentido emplear dicho parámetro.

Por su parte, el parámetro ramdom_state permite obtener la misma partición cada vez que se ejecuta el script.

# Actividad 2:    

Entrenar y evaluar con al menos 3 nuevos modelos (Sugerencias: SVR , RandomForestRegressor, GradientBoostingRegressor, etc.) Obligatorio: Probar con una red neuronal. Puede ser de scikit-learn o de alguna otra librería que deseen como keras , pytorch , etc.). Junto con las métricas debe entregarse una breve descripción de cómo funciona cada modelo. Importante: Para evaluar, por ejemplo mean_absolute_error provee un parámetro multioutput que debería tomar el valor ` raw_values ` para reportar métricas para cada dimension de output por separado.

In [11]:
MAE = make_scorer(mean_absolute_error,greater_is_better=False)

### Random Forest Regressor

In [12]:
rfr = RandomForestRegressor(n_estimators=10, n_jobs=1, random_state=42, criterion='mae', max_depth=20, max_features =100)
rfr.fit(X_train, y_train)    
y_pred_rfr=rfr.predict(X_test) 
mean_absolute_error(y_test, y_pred_rfr, multioutput='raw_values') 

array([1444.73372966,   10.7539174 ,    7.92150188,    6.35844806])

In [13]:
#fitRFR=rfr.fit(X_train, y_train)  
#from pprint import pprint
#pprint(vars(fitRFR))
#dir(fitRFR)
#getattr(fitRFR, 'feature_importances_')

### Gradient Boosting Regressor

In [14]:
gbr = GradientBoostingRegressor(n_estimators=10, random_state=42, criterion='mae', max_features=100, loss='lad')
gbr.fit(X_train, y_train.iloc[:,0])   
y_pred_gbr=gbr.predict(X_test)
y_pred_train = gbr.predict(X_train)
print('MAE en entrenamiento')
print(mean_absolute_error(y_train.iloc[:,0], y_pred_train))
print('MAE en test')
print(mean_absolute_error(y_test.iloc[:,0], y_pred_gbr))

MAE en entrenamiento
1723.1881163970709
MAE en test
1610.587101927204


In [15]:
#gbr.fit(X_train, y_train.iloc[:,0])    

In [16]:
#y_pred_gbr=gbr.predict(X_test) 
#mean_absolute_error(y_test.iloc[:,0], y_pred_gbr)

Cuando se intenta estimar las cuatro variables en forma simultánea se obtiene un mensaje de error, lo que indica que probablemente el método solo funciona con una etiqueta. Por eso, por el momento solo se estima una de las variables SHP.

### Red neuronal (MLP)

In [18]:
reg = MLPRegressor(hidden_layer_sizes=(10,),  activation='relu', solver='adam',    alpha=0.001,batch_size='auto',
               learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
               random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9,
               nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999,
               epsilon=1e-08)

reg = reg.fit(X_train, y_train)

In [19]:
y_pred_mlp = reg.predict(X_test)
mean_absolute_error(y_test, y_pred_mlp, multioutput='raw_values') 

array([1763.47306172,   11.6208516 ,    8.27505239,    6.62763106])

En estos modelos base sin optimizar hiperparámetros, tiene un desempeño superior la regresión random forest.
En el caso de la regresión gradient boosting no se pudo implementar estimaciones para más de una etiqueta en forma simultánea.

# Actividad 3

Para estos nuevos modelos tunear hiper-parámetros. Para las evaluaciones utilizar la técnica de k-fold cross-validation (ver cross-validation ) y explicar los resultados.

En esta actividad directamente se tunean los hiperparámetros y se comparan los modelos de acuerdo al MAE. 
No se realiza la actividad 4, porque el score F1 no aplica a problemas de regresión. Se suple eas actividad justamente por medio del uso del MAE en esta actividad.

In [20]:
# definición de métrica para usar con GridSearchCV
MAE = make_scorer(mean_absolute_error,greater_is_better=False)
# no se puede usar el argumento multioutput con el setting "raw_values" porque se el scorer debe ser un único número

### Random Forest Regressor

In [21]:
def random_forest_forecast(X_train,X_test,y_train, y_test):
    
    rfr = RandomForestRegressor(n_jobs=1, random_state=0)
    param_grid = {'n_estimators': [10,20, 50], 'max_depth':[10,20,50]}
    # 'n_estimators': [1000], 'max_features': [10,15,20,25], 'max_depth':[20,20,25,25,]}
    model = GridSearchCV(estimator=rfr, param_grid=param_grid, n_jobs=1, cv=10, scoring=MAE)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    
    print('Random forest regression...')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train, y_pred_train, multioutput= 'raw_values'))
    print('MAE en test:')
    print(mean_absolute_error(y_test, y_pred, multioutput= 'raw_values'))

In [22]:
random_forest_forecast(X_train,X_test,y_train, y_test)

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Random forest regression...
Mejores Parámetros:
{'max_depth': 50, 'n_estimators': 50}
Best CV Score:
408.91731615925863
MAE en entrenamiento:
[876.55765187   7.99337243   5.85359339   4.58556026]
MAE en test:
[1518.36575481   10.60968949    7.85397689    6.30025063]


El modelo sobreajusta demasiado, porque el desempeño en test es muy inferior al desempeño en la muestra de entrenamiento. Es intrigante que empeora el modelo al seleccionar los hiperparámetos por cross validation ( en el modelo original sin tunear hiperparámetros se tenía un mejor desempeño en test.

### Gradient Boosting Regression

In [23]:
def gradient_boosting_forecast(X_train,X_test,y_train, y_test):
    
    gbr = GradientBoostingRegressor(loss='lad', random_state=5,criterion='mae')
    param_grid = {'n_estimators': [10], 'learning_rate':[0.01,0.1,1,10]}
    # 'n_estimators': [10], 'max_features': [10,15,20,25], 'learning_rate':[0.01,0.1,1,10]}
    model = GridSearchCV(estimator=gbr, param_grid=param_grid,  cv=10)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    
    print('Gradient boosting regression...')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train, y_pred_train, multioutput= 'raw_values'))
    print('MAE en test:')
    print(mean_absolute_error(y_test, y_pred, multioutput= 'raw_values'))

In [24]:
gradient_boosting_forecast(X_train,X_test,y_train.iloc[:,0], y_test.iloc[:,0])

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Gradient boosting regression...
Mejores Parámetros:
{'learning_rate': 1, 'n_estimators': 10}
Best CV Score:
-0.10258190582219315
MAE en entrenamiento:
[1515.22925149]
MAE en test:
[1453.32665832]


### Red neuronal (MLP)

In [25]:
def multi_layer_perceptron_forecast(X_train,X_test,y_train, y_test):
    
    mlp = MLPRegressor()
    param_grid = {'solver': ['lbfgs'], 'max_iter': [1000 ], 'alpha': 10.0 ** -np.arange(1, 2), 
                  'hidden_layer_sizes':10*np.arange(10, 15), 'random_state':[5]}
    model = GridSearchCV(estimator=mlp, param_grid=param_grid,  cv=10)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    
    print('Multi-layer perceptron regression...')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train, y_pred_train, multioutput= 'raw_values'))
    print('MAE en test:')
    print(mean_absolute_error(y_test, y_pred, multioutput= 'raw_values'))

In [26]:
multi_layer_perceptron_forecast(X_train,X_test,y_train, y_test)

C:\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_

C:\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_

C:\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_

C:\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_

Multi-layer perceptron regression...
Mejores Parámetros:
{'alpha': 0.1, 'hidden_layer_sizes': 110, 'max_iter': 1000, 'random_state': 5, 'solver': 'lbfgs'}
Best CV Score:
-0.15458933982650958
MAE en entrenamiento:
[1959.21856534   45.38964898   31.67319572   17.10271971]
MAE en test:
[1836.9966037    45.63799425   31.94461493   16.2676895 ]


Se tiene un desempeño marcadamente inferior al de las regresiones random forest y gradient boosting. Además es mejor el ajuste en test que en entenamiento. Por lo tanto probablemente se deba probar un arquitectura mas compleja.

# Actividad 4

Elegir el mejor modelo entrenado hasta el momento según f1-score. Comparar las métricas de este modelo vs. las métricas de evaluar 4 modelos por separado, un modelo para cada uno de los targets. Interpretar los resultados. 

No se emplea el f1-score porque no aplica a problemas de regresión. En la actividad 3 se computaron las métricas MAE, para cada uno de los modelos seleccionados, dando mejores resultados la regresión random forest. En esta actividad se agregan las métricas que se obtienen al estimar modelos para cada variable por separado.

## Regresiones con un solo output

### Regresiones random forest con un output

In [27]:
def univariate_random_forest_forecast(X_train,X_test,y_train, y_test):
    
    rfr = RandomForestRegressor(n_jobs=1, random_state=0)
    param_grid = {'n_estimators': [10,20, 50], 'max_depth':[10,20,50]}
    # 'n_estimators': [1000], 'max_features': [10,15,20,25], 'max_depth':[20,20,25,25,]}
    model = GridSearchCV(estimator=rfr, param_grid=param_grid, n_jobs=1, cv=10, scoring=MAE)
    model.fit(X_train, y_train.iloc[:,0])
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    
    print('Random forest regression para SHP_WEIGHT')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train.iloc[:,0], y_pred_train))
    print('MAE en test:')
    print(mean_absolute_error(y_test.iloc[:,0], y_pred))
    
    rfr = RandomForestRegressor(n_jobs=1, random_state=0)
    param_grid = {'n_estimators': [10,20, 50], 'max_depth':[10,20,50]}
    # 'n_estimators': [1000], 'max_features': [10,15,20,25], 'max_depth':[20,20,25,25,]}
    model = GridSearchCV(estimator=rfr, param_grid=param_grid, n_jobs=1, cv=10, scoring=MAE)
    model.fit(X_train, y_train.iloc[:,1])
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    
    print('Random forest regression para SHP_LENGTH')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train.iloc[:,1], y_pred_train))
    print('MAE en test:')
    print(mean_absolute_error(y_test.iloc[:,1], y_pred))
    
    rfr = RandomForestRegressor(n_jobs=1, random_state=0)
    param_grid = {'n_estimators': [10,20, 50], 'max_depth':[10,20,50]}
    # 'n_estimators': [1000], 'max_features': [10,15,20,25], 'max_depth':[20,20,25,25,]}
    model = GridSearchCV(estimator=rfr, param_grid=param_grid, n_jobs=1, cv=10, scoring=MAE)
    model.fit(X_train, y_train.iloc[:,2])
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    
    print('Random forest regression para SHP_WIDTH')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train.iloc[:,2], y_pred_train))
    print('MAE en test:')
    print(mean_absolute_error(y_test.iloc[:,2], y_pred))
    
    rfr = RandomForestRegressor(n_jobs=1, random_state=0)
    param_grid = {'n_estimators': [10,20, 50], 'max_depth':[10,20,50]}
    # 'n_estimators': [1000], 'max_features': [10,15,20,25], 'max_depth':[20,20,25,25,]}
    model = GridSearchCV(estimator=rfr, param_grid=param_grid, n_jobs=1, cv=10, scoring=MAE)
    model.fit(X_train, y_train.iloc[:,3])
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    
    print('Random forest regression para SHP_HEIGHT')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train.iloc[:,3], y_pred_train))
    print('MAE en test:')
    print(mean_absolute_error(y_test.iloc[:,3], y_pred))

In [28]:
univariate_random_forest_forecast(X_train,X_test,y_train, y_test)

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Random forest regression para SHP_WEIGHT
Mejores Parámetros:
{'max_depth': 50, 'n_estimators': 50}
Best CV Score:
1610.155447361426
MAE en entrenamiento:
876.0011442243941
MAE en test:
1513.4209800097108
Random forest regression para SHP_LENGTH
Mejores Parámetros:
{'max_depth': 50, 'n_estimators': 50}
Best CV Score:
11.050254622451193
MAE en entrenamiento:
7.006184542185479
MAE en test:
10.659708741151473
Random forest regression para SHP_WIDTH
Mejores Parámetros:
{'max_depth': 50, 'n_estimators': 20}
Best CV Score:
7.842746099022772
MAE en entrenamiento:
5.012509491781855
MAE en test:
7.768557035037964
Random forest regression para SHP_HEIGHT
Mejores Parámetros:
{'max_depth': 50, 'n_estimators': 50}
Best CV Score:
6.069047992965159
MAE en entrenamiento:
4.008781697138562
MAE en test:
6.063754965977446


El ajuste de los modelos univariados es mejor que el del modelo multivariante para SHP_WEIGHT y SHP_LENGTH, ocurriendo lo contrario para SHP_WIDTH y SHP_HEIGHT.

### Regresiones gradient boosting con un output

In [29]:
def univariate_gradient_boosting_forecast(X_train,X_test,y_train, y_test):
    
    gbr = GradientBoostingRegressor(loss='lad', random_state=5,criterion='mae')
    param_grid = {'n_estimators': [10], 'learning_rate':[0.01,0.1,1,10]}
    # 'n_estimators': [10], 'max_features': [10,15,20,25], 'learning_rate':[0.01,0.1,1,10]}
    model = GridSearchCV(estimator=gbr, param_grid=param_grid,  cv=10)
    model.fit(X_train, y_train.iloc[:,0])
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    
    print('Gradient boosting regression para SHP_WEIGHT')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train.iloc[:,0], y_pred_train))
    print('MAE en test:')
    print(mean_absolute_error(y_test.iloc[:,0], y_pred))
    
    gbr = GradientBoostingRegressor(loss='lad', random_state=5,criterion='mae')
    param_grid = {'n_estimators': [10], 'learning_rate':[0.01,0.1,1,10]}
    # 'n_estimators': [10], 'max_features': [10,15,20,25], 'learning_rate':[0.01,0.1,1,10]}
    model = GridSearchCV(estimator=gbr, param_grid=param_grid,  cv=10)
    model.fit(X_train, y_train.iloc[:,1])
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    
    print('Gradient boosting regression para SHP_LENGTH')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train.iloc[:,1], y_pred_train))
    print('MAE en test:')
    print(mean_absolute_error(y_test.iloc[:,1], y_pred))
    
    gbr = GradientBoostingRegressor(loss='lad', random_state=5,criterion='mae')
    param_grid = {'n_estimators': [10], 'learning_rate':[0.01,0.1,1,10]}
    # 'n_estimators': [10], 'max_features': [10,15,20,25], 'learning_rate':[0.01,0.1,1,10]}
    model = GridSearchCV(estimator=gbr, param_grid=param_grid,  cv=10)
    model.fit(X_train, y_train.iloc[:,2])
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    
    print('Gradient boosting regression para SHP_WIDTH')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train.iloc[:,2], y_pred_train))
    print('MAE en test:')
    print(mean_absolute_error(y_test.iloc[:,2], y_pred))
    
    gbr = GradientBoostingRegressor(loss='lad', random_state=5,criterion='mae')
    param_grid = {'n_estimators': [10], 'learning_rate':[0.01,0.1,1,10]}
    # 'n_estimators': [10], 'max_features': [10,15,20,25], 'learning_rate':[0.01,0.1,1,10]}
    model = GridSearchCV(estimator=gbr, param_grid=param_grid,  cv=10)
    model.fit(X_train, y_train.iloc[:,3])
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    
    print('Gradient boosting regression para SHP_HEIGHT')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train.iloc[:,3], y_pred_train))
    print('MAE en test:')
    print(mean_absolute_error(y_test.iloc[:,3], y_pred))

In [30]:
univariate_gradient_boosting_forecast(X_train,X_test,y_train, y_test)

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Gradient boosting regression para SHP_WEIGHT
Mejores Parámetros:
{'learning_rate': 1, 'n_estimators': 10}
Best CV Score:
-0.10258190582219315
MAE en entrenamiento:
1515.2292514876292
MAE en test:
1453.3266583229035


C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Gradient boosting regression para SHP_LENGTH
Mejores Parámetros:
{'learning_rate': 1, 'n_estimators': 10}
Best CV Score:
-0.06265806150027763
MAE en entrenamiento:
10.465988098966488
MAE en test:
10.332290362953692
Gradient boosting regression para SHP_WIDTH
Mejores Parámetros:
{'learning_rate': 1, 'n_estimators': 10}
Best CV Score:
-0.06006657216231128
MAE en entrenamiento:
7.670435327278421
MAE en test:
7.663078848560701


C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Gradient boosting regression para SHP_HEIGHT
Mejores Parámetros:
{'learning_rate': 1, 'n_estimators': 10}
Best CV Score:
-0.0674301968190712
MAE en entrenamiento:
5.802912621359224
MAE en test:
6.170337922403005


Por lo general es mejor el desempeño de Gradient Boosting que Random Forest. En casi todos los casos tiene mejor ajuste en test y sobrejusta notablemente menos en el conjunto de entrenamiento.

### Regresiones MLP con un output

In [12]:
def univariate_multi_layer_perceptron_forecast(X_train,X_test,y_train, y_test):
    
    mlp = MLPRegressor()
    param_grid = {'solver': ['lbfgs'], 'max_iter': [1000 ], 'alpha': 10.0 ** -np.arange(1, 2), 
                  'hidden_layer_sizes':10*np.arange(10, 15), 'random_state':[5]}
    model = GridSearchCV(estimator=mlp, param_grid=param_grid,  cv=10)
    model.fit(X_train, y_train.iloc[:,0])
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    print('Multi-layer perceptron regression para SHP_WEIGHT')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train.iloc[:,0], y_pred_train))
    print('MAE en test:')
    print(mean_absolute_error(y_test.iloc[:,0], y_pred))
    
    mlp = MLPRegressor()
    param_grid = {'solver': ['lbfgs'], 'max_iter': [1000 ], 'alpha': 10.0 ** -np.arange(1, 2), 
                  'hidden_layer_sizes':10*np.arange(10, 15), 'random_state':[5]}
    model = GridSearchCV(estimator=mlp, param_grid=param_grid,  cv=10)
    model.fit(X_train, y_train.iloc[:,1])
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    print('Multi-layer perceptron regression para SHP_LENGTH')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train.iloc[:,1], y_pred_train))
    print('MAE en test:')
    print(mean_absolute_error(y_test.iloc[:,1], y_pred))
    
    mlp = MLPRegressor()
    param_grid = {'solver': ['lbfgs'], 'max_iter': [1000 ], 'alpha': 10.0 ** -np.arange(1, 2), 
                  'hidden_layer_sizes':10*np.arange(10, 15), 'random_state':[5]}
    model = GridSearchCV(estimator=mlp, param_grid=param_grid,  cv=10)
    model.fit(X_train, y_train.iloc[:,2])
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    print('Multi-layer perceptron regression para SHP_WIDTH')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train.iloc[:,2], y_pred_train))
    print('MAE en test:')
    print(mean_absolute_error(y_test.iloc[:,2], y_pred))
    
    mlp = MLPRegressor()
    param_grid = {'solver': ['lbfgs'], 'max_iter': [1000 ], 'alpha': 10.0 ** -np.arange(1, 2), 
                  'hidden_layer_sizes':10*np.arange(10, 15), 'random_state':[5]}
    model = GridSearchCV(estimator=mlp, param_grid=param_grid,  cv=10)
    model.fit(X_train, y_train.iloc[:,3])
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    print('Multi-layer perceptron regression para SHP_HEIGHT')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train.iloc[:,3], y_pred_train))
    print('MAE en test:')
    print(mean_absolute_error(y_test.iloc[:,3], y_pred))

In [ ]:
univariate_multi_layer_perceptron_forecast(X_train,X_test,y_train, y_test)

Multi-layer perceptron regression para SHP_WEIGHT
Mejores Parámetros:
{'alpha': 0.1, 'hidden_layer_sizes': 120, 'max_iter': 1000, 'random_state': 5, 'solver': 'lbfgs'}
Best CV Score:
-0.18575144540361502
MAE en entrenamiento:
1901.349860132466
MAE en test:
1778.9600627448278
Multi-layer perceptron regression para SHP_LENGTH
Mejores Parámetros:
{'alpha': 0.1, 'hidden_layer_sizes': 100, 'max_iter': 1000, 'random_state': 5, 'solver': 'lbfgs'}
Best CV Score:
-0.10325982423323989
MAE en entrenamiento:
11.903950526753693
MAE en test:
11.5657392216161


In [ ]:
Las regresiones MLP univariates funcionan un poco mejor que la multivariante.

# Extra: búsqueda de una mejor arquitectura

En las actividades previas se empleó MLP con una sola capa. En este apartado se escribe código para luego (con más poder de cómputo) construir una arquitectura más compleja. Para compensar el problema de carga computacional extra se reducen la cantidad de iteraciones máximas y el parámétro de cross validation.También se usa un único learning rate.

In [ ]:
def deep_MLP(X_train,X_test,y_train, y_test):
    
    mlp = MLPRegressor(random_state=5, max_iter=100, alpha=0.1, solver='adam')
    param_grid = {'hidden_layer_sizes':[(100,50,25), (120,60,30),(50,25,10)]}
    model = GridSearchCV(estimator=mlp, param_grid=param_grid,  cv=3)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    
    print('Multi-layer perceptron regression...')
    print('Mejores Parámetros:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)
    print('MAE en entrenamiento:')
    print(mean_absolute_error(y_train, y_pred_train, multioutput= 'raw_values'))
    print('MAE en test:')
    print(mean_absolute_error(y_test, y_pred, multioutput= 'raw_values'))

In [ ]:
deep_MLP(X_train,X_test,y_train, y_test)